__Step 1__Model-based feature selection (SelectFromModel)

        https://www.kaggle.com/residentmario/automated-feature-selection-with-sklearn



   __Step 1.1__Estimating Regression Model and assessing overall model fit.

    This stage includes:

    -- Select method for specifying the Regression Model. This can be done using:

        Stepwise Regression:--  
        Forward Selection - Backward Elimination
    
        Instead of manually selecting the variables, we can automate this process by using forward or backward selection. 
        Forward selection starts with most significant predictor in the model and adds variable for each step. Backward 
        elimination starts with all predictors in the model and removes the least significant variable for each step. 
    
__Step 2__Examine the statistical significance of the model by checking:

 -- Selecting criteria can be set to any statistical measure like R-square, t-stat etc.

 -- anova(analysis of variance) -(https://pythonfordatascience.org/anova-python/)

 -- F-Statistic -  An F test tests a model which may have multiple explanatory variables contributing to the variance explained 
     by the model.

 -- Coefficient of determination - R2 & Adjusted R2

 -- Residual Standard Error /Standard error of estimate (Se)
    https://www.analyticsvidhya.com/blog/2017/06/a-comprehensive-guide-for-linear-ridge-and-lasso-regression/
    
 -- Statistical significance of the estimated parameters using t-test

__Step 3__Test the Regression Variable for meeting regression assumptions.

 -- Use residual plots to test the assumptions of: linearity, homoscedasticity, independence and normality.
    https://www.analyticsvidhya.com/blog/2017/06/a-comprehensive-guide-for-linear-ridge-and-lasso-regression/

__Step 4__Perform Enhanced Regression diagnostics 

- Run the omnibus test gvlma(fit) that performs a global validation of linear model assumptions 

__Step 5__Identify the Influential Observations